In [1]:
json_mcp = """{
  "mcpServers": {
    "infobip-sms": {
      "command": "uvx",
      "args": [
        "mcp-proxy",
        "http://mcp.astha.ai/sse",
        "--headers",
        "X-User-Access-Token",
        "qLUOhv+OUxKsdtvX6HZBVw==.miU1K6bmQXCrJ8M1rKFFeM6ItFm3IBKP+vEoNT+SBRecKTia/yPnX68zEYx/J31Oj2OJjfwixzZ8ElI9P9bErVCPbxXfhTR+QPNeKzlkso5IVULnsv6XlTlpW94VlBG7S07JEm3Itq+cDE9KQwQeEA==",
        "--headers",
        "X-Aztp-ID",
        "aztp://aztp.network/workload/development/node/sunday-hackathon/infobip-sms"
      ]
    }
  }
}"""

In [2]:
import json

json_data = json.loads(json_mcp)

# save json_data to a file
config_file_path = "test_json.json"
with open(config_file_path, 'w') as f:
    json.dump(json_data, f)

user_access_token = json_data['mcpServers']['infobip-sms']['args'][4]
aztp_id = json_data['mcpServers']['infobip-sms']['args'][7]

In [3]:
user_access_token ,aztp_id

('qLUOhv+OUxKsdtvX6HZBVw==.miU1K6bmQXCrJ8M1rKFFeM6ItFm3IBKP+vEoNT+SBRecKTia/yPnX68zEYx/J31Oj2OJjfwixzZ8ElI9P9bErVCPbxXfhTR+QPNeKzlkso5IVULnsv6XlTlpW94VlBG7S07JEm3Itq+cDE9KQwQeEA==',
 'aztp://aztp.network/workload/development/node/sunday-hackathon/infobip-sms')

In [4]:
from get_rbac_policy import fetch_rbac_policy_async

policy_async = await fetch_rbac_policy_async(user_access_token, aztp_id)

# Display key information
print(f"✓ Policy: {policy_async.get('metadata', {}).get('name')}")
print(f"✓ MCP URL: {policy_async.get('mcp_url')}")
print(f"✓ Allowed tools: {policy_async.get('tools', {}).get('allowed_tools')}")

Fetching policy from https://api.astha.ai/astha/v1/user-workload/access-token/rbac for aztp_id=aztp://aztp.network/workload/development/node/sunday-hackathon/infobip-sms
✓ Successfully fetched and cached policy for aztp_id=aztp://aztp.network/workload/development/node/sunday-hackathon/infobip-sms
✓ Policy version: 1.0
✓ Policy name: User RBAC Policy
✓ MCP URL: https://mcp.infobip.com/sms/sse
✓ Allowed tools: 5
  Tools: get_scheduled_sms_messages, get_scheduled_sms_messages_status, get_sms_message_delivery_reports, get_sms_message_logs, get_current_server_utc_time
✓ Denied tools: 4
✓ Policy: User RBAC Policy
✓ MCP URL: https://mcp.infobip.com/sms/sse
✓ Allowed tools: ['get_scheduled_sms_messages', 'get_scheduled_sms_messages_status', 'get_sms_message_delivery_reports', 'get_sms_message_logs', 'get_current_server_utc_time']


In [ ]:
import os
from langchain_anthropic import ChatAnthropic

In [ ]:
# # Check for required environment variables
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError(
        "ANTHROPIC_API_KEY environment variable is not set. "
        "Please set it in your .env file or environment."
    )


In [8]:
from mcp_use import MCPClient, MCPAgent
# Create MCP client from config file
try:
    client = MCPClient.from_config_file(config_file_path)
except Exception as e:
    print(f"Failed to create MCP client: {str(e)}")

# Create sessions for all servers
try:
    await client.create_all_sessions()
    print("Created MCP sessions for all servers")
except Exception as e:
    print(f"Failed to create MCP sessions: {str(e)}")

2025-12-29 16:01:12,623 - mcp_use.telemetry.telemetry - INFO - Anonymized telemetry enabled. Set MCP_USE_ANONYMIZED_TELEMETRY=false to disable.
Created MCP sessions for all servers


In [9]:
# Create LLM and agent with memory enabled for session persistence

llm = ChatAnthropic(
    model=os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-5-20250929"),
    api_key=api_key
)

agent = MCPAgent(
    llm=llm,
    client=client,
    max_steps=15,
    memory_enabled=True,  # Enable memory for session persistence
)
print("MCP agent initialized with memory enabled")

MCP agent initialized with memory enabled


In [10]:
await agent.run("Hello, list all the tools?")

2025-12-29 16:01:16,904 - mcp_use - INFO - 🚀 Initializing MCP agent and connecting to services...
2025-12-29 16:01:16,905 - mcp_use - INFO - 🔌 Found 1 existing sessions
2025-12-29 16:01:18,719 - mcp_use - INFO - 🛠️ Created 5 LangChain tools from client
2025-12-29 16:01:18,720 - mcp_use - INFO - 🧰 Found 5 tools across all connectors
2025-12-29 16:01:18,721 - mcp_use - INFO - 🧠 Agent ready with tools: get_scheduled_sms_messages, get_scheduled_sms_messages_status, get_sms_message_delivery_reports, get_sms_message_logs, get_current_server_utc_time
2025-12-29 16:01:18,737 - mcp_use - INFO - ✨ Agent initialization complete
2025-12-29 16:01:18,738 - mcp_use - INFO - 💬 Received query: 'Hello, list all the tools?'
2025-12-29 16:01:18,739 - mcp_use - INFO - 🏁 Starting agent execution
2025-12-29 16:01:25,811 - mcp_use - INFO - ✅ Agent finished with output
2025-12-29 16:01:25,813 - mcp_use - INFO - 🎉 Agent execution complete in 8.91 seconds


"Hello! Here are all the tools I have access to:\n\n1. **get_scheduled_sms_messages** - Retrieves all scheduled SMS messages and their scheduled date and time for a given bulk ID.\n\n2. **get_scheduled_sms_messages_status** - Gets the status of scheduled SMS messages for a given bulk ID.\n\n3. **get_sms_message_delivery_reports** - Fetches batches of SMS message delivery reports to confirm whether specific messages have been delivered. Useful when you can't receive real-time delivery reports.\n\n4. **get_sms_message_logs** - Obtains logs associated with outbound SMS messages. Limited to logs from the last 48 hours with a maximum of 1000 logs per call.\n\n5. **get_current_server_utc_time** - Returns the current UTC date and time. Useful for scheduling messages to be sent at a specific time in the future.\n\nThese tools allow you to manage and monitor SMS messages, including checking delivery status, viewing logs, scheduling messages, and checking scheduled message details.\n\nIs there a